# DC Inside 만갤 개념글 db화
- 주의 : 불법 번역 만화를 찾는 용도입니다. (현황 분석용)
- DCinsice robots 정책

```
User-agent: *
Disallow: /

# Ads
User-agent: grapeshot
Allow: /  

# Search
User-agent: Googlebot
Allow : / 
Crawl-delay: 60
```

In [4]:
!scrapy startproject comics

Error: scrapy.cfg already exists in C:\Users\yoon hwa\Documents\dev\Crawling_Project\comics


In [5]:
!tree comics

폴더 PATH의 목록입니다.
볼륨 일련 번호는 A876-8D28입니다.
C:\USERS\YOON HWA\DOCUMENTS\DEV\CRAWLING_PROJECT\COMICS
└─comics
    ├─spiders
    │  └─__pycache__
    └─__pycache__


list + contents

### 0. xpath 추출하기
- dc inside는 fake_useragent가 필요하다.

In [3]:
import scrapy
import requests
from scrapy.http import TextResponse

In [46]:
from fake_useragent import UserAgent
UserAgent().chrome

'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.0 Safari/537.36'

In [52]:
page = 1
headers = {
    "user-agent":UserAgent().chrome
}
req = requests.get(url="https://gall.dcinside.com/board/lists/?id=comic_new2&page={}&exception_mode=recommend".format(page), headers=headers)

In [53]:
response = TextResponse(req.url, body=req.text, encoding="utf-8")

In [54]:
req, response

(<Response [200]>,
 <200 https://gall.dcinside.com/board/lists/?id=comic_new2&page=1&exception_mode=recommend>)

In [155]:
# links
links = response.xpath('//*[@class="ub-content us-post"]/td[2]/a[1]/@href').extract()

In [156]:
len(links)

50

In [162]:
link = response.urljoin(links[0])
link

'https://gall.dcinside.com/board/view/?id=comic_new2&no=1085660&exception_mode=recommend&page=1'

In [163]:
req = requests.get(url=link, headers=headers)

In [164]:
response = TextResponse(req.url, body=req.text, encoding="utf-8")

In [167]:
response

<200 https://gall.dcinside.com/board/view/?id=comic_new2&no=1085660&exception_mode=recommend&page=1>

In [181]:
# title
title = response.xpath('//*[@class="gallview_head clear ub-content"]/h3/span[2]/text()').extract_first()
title

'번역)LV1 마왕과 원룸 용사 9-2화(빛나랑 골덕 작가)'

In [205]:
# views
views = response.xpath('//*[@class="fr"]/span[1]/text()').extract_first()
views[3:]

'4173'

In [206]:
# date
date = response.xpath('//*[@class="fl"]/span[2]/text()').extract_first()
date

'2019.12.10 23:58:07'

In [198]:
# recommend
recommend = response.xpath('//*[@class="gall_reply_num"]/text()').extract_first()
recommend[3:]

'222'

In [201]:
# 이미지 개수
imgs = response.xpath('//*[@style="overflow:hidden;"]/a/@href').extract()
len(imgs)

8

In [202]:
# 첫번째 이미지 링크
imgs[0]

'https://2.bp.blogspot.com/-n-Pk3MTs1oE/Xe-xtRtpyJI/AAAAAAAAFO8/lVeiuVghkrMeSm5sNLCEL5c5Zhz6bvdVQCLcBGAsYHQ/s5000/9-015.png'

### 1. items.py 만들기

In [210]:
%%writefile comics/comics/items.py

# Define here the models for your scraped items
#
# See documentation in:
# https://docs.scrapy.org/en/latest/topics/items.html

import scrapy


class ComicsItem(scrapy.Item):
    # define the fields for your item here like:
    # name = scrapy.Field()
    title = scrapy.Field()
    date = scrapy.Field()
    views = scrapy.Field()
    recommend = scrapy.Field()
    link = scrapy.Field()
    img_count = scrapy.Field()
    img_link = scrapy.Field()

Overwriting comics/comics/items.py


### 2. spider 만들기
- error : 이미지가 없는 경우

In [226]:
%%writefile comics/comics/spiders/spider.py
import scrapy
from comics.items import ComicsItem

class ComicsSpider(scrapy.Spider):
    name = "Comics"
    custom_settings = {
        'DOWNLOADER_MIDDLEWARES' : {
            'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware': None,
            'scrapy_fake_useragent.middleware.RandomUserAgentMiddleware': 400,
        }
    }
    
    def __init__(self, pages=1, **kwargs):
        self.start_url = "https://gall.dcinside.com/board/lists/?id=comic_new2&page={}&exception_mode=recommend".format(pages)
        super().__init__(**kwargs)
        
    def start_requests(self):
        url = self.start_url
        yield scrapy.Request(url, callback=self.parse)
        
    def parse(self, response):
        links = response.xpath('//*[@class="ub-content us-post"]/td[2]/a[1]/@href').extract()
        links = list(map(response.urljoin, links))
        for link in links:
            yield scrapy.Request(link, callback=self.page_parse)
    
    def page_parse(self, response):
        item = ComicsItem()
        item["title"] = response.xpath('//*[@class="gallview_head clear ub-content"]/h3/span[2]/text()').extract_first()
        item["date"] = response.xpath('//*[@class="fl"]/span[2]/text()').extract_first()
        item["views"] = response.xpath('//*[@class="fr"]/span[1]/text()').extract_first()[3:]
        item["recommend"] = response.xpath('//*[@class="gall_reply_num"]/text()').extract_first()[3:]
        item["link"] = response.url
        try:
            item["img_count"] = len(response.xpath('//*[@style="overflow:hidden;"]/a/@href').extract())
            item["img_link"] = response.xpath('//*[@style="overflow:hidden;"]/a/@href').extract()[0]
        except:
            item["img_count"] = "0"
            item["img_link"] = "0"
        yield item

Overwriting comics/comics/spiders/spider.py


### 3. pipeline for mongodb

In [225]:
%%writefile comics/comics/mongodb.py
import pymongo

client = pymongo.MongoClient('mongodb://15.165.28.220:27017/') # mongodb ip 필요
db = client.comics
collection = db.bestcomiecs
# mongodb 모듈 생성

Overwriting comics/comics/mongodb.py


In [217]:
%%writefile comics/comics/pipelines.py
from .mongodb import collection

class ComicsPipeline(object):

    def process_item(self, item, spider):
        
        data = {
            "title": item["title"], 
            "date": item["date"],
            "views": item["views"], 
            "recommend": item["recommend"],
            "link": item["link"],
            "img_count": item["img_count"],
            "img_link": item["img_link"],
               }
        collection.insert(data)
        return item

Overwriting comics/comics/pipelines.py


### 4. setting.py 변경
- ROBOTSTXT_OBEY = False
- ITEM_PIPELINES = 300
- DOWNLOAD_DELAY = 60 -> 1 : 최대 속도로 조정 필요

In [227]:
%%writefile comics/comics/settings.py

# Scrapy settings for comics project
#
# For simplicity, this file contains only settings considered important or
# commonly used. You can find more settings consulting the documentation:
#
#     https://docs.scrapy.org/en/latest/topics/settings.html
#     https://docs.scrapy.org/en/latest/topics/downloader-middleware.html
#     https://docs.scrapy.org/en/latest/topics/spider-middleware.html

BOT_NAME = 'comics'

SPIDER_MODULES = ['comics.spiders']
NEWSPIDER_MODULE = 'comics.spiders'


# Crawl responsibly by identifying yourself (and your website) on the user-agent
#USER_AGENT = 'comics (+http://www.yourdomain.com)'

# Obey robots.txt rules
ROBOTSTXT_OBEY = False

# Configure maximum concurrent requests performed by Scrapy (default: 16)
#CONCURRENT_REQUESTS = 32

# Configure a delay for requests for the same website (default: 0)
# See https://docs.scrapy.org/en/latest/topics/settings.html#download-delay
# See also autothrottle settings and docs
## DOWNLOAD_DELAY = 60
# The download delay setting will honor only one of:
#CONCURRENT_REQUESTS_PER_DOMAIN = 16
#CONCURRENT_REQUESTS_PER_IP = 16

# Disable cookies (enabled by default)
#COOKIES_ENABLED = False

# Disable Telnet Console (enabled by default)
#TELNETCONSOLE_ENABLED = False

# Override the default request headers:
#DEFAULT_REQUEST_HEADERS = {
#   'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
#   'Accept-Language': 'en',
#}

# Enable or disable spider middlewares
# See https://docs.scrapy.org/en/latest/topics/spider-middleware.html
#SPIDER_MIDDLEWARES = {
#    'comics.middlewares.ComicsSpiderMiddleware': 543,
#}

# Enable or disable downloader middlewares
# See https://docs.scrapy.org/en/latest/topics/downloader-middleware.html
#DOWNLOADER_MIDDLEWARES = {
#    'comics.middlewares.ComicsDownloaderMiddleware': 543,
#}

# Enable or disable extensions
# See https://docs.scrapy.org/en/latest/topics/extensions.html
#EXTENSIONS = {
#    'scrapy.extensions.telnet.TelnetConsole': None,
#}

# Configure item pipelines
# See https://docs.scrapy.org/en/latest/topics/item-pipeline.html
ITEM_PIPELINES = {
    'comics.pipelines.ComicsPipeline': 300,
}

# Enable and configure the AutoThrottle extension (disabled by default)
# See https://docs.scrapy.org/en/latest/topics/autothrottle.html
#AUTOTHROTTLE_ENABLED = True
# The initial download delay
#AUTOTHROTTLE_START_DELAY = 5
# The maximum download delay to be set in case of high latencies
#AUTOTHROTTLE_MAX_DELAY = 60
# The average number of requests Scrapy should be sending in parallel to
# each remote server
#AUTOTHROTTLE_TARGET_CONCURRENCY = 1.0
# Enable showing throttling stats for every response received:
#AUTOTHROTTLE_DEBUG = False

# Enable and configure HTTP caching (disabled by default)
# See https://docs.scrapy.org/en/latest/topics/downloader-middleware.html#httpcache-middleware-settings
#HTTPCACHE_ENABLED = True
#HTTPCACHE_EXPIRATION_SECS = 0
#HTTPCACHE_DIR = 'httpcache'
#HTTPCACHE_IGNORE_HTTP_CODES = []
#HTTPCACHE_STORAGE = 'scrapy.extensions.httpcache.FilesystemCacheStorage'

Overwriting comics/comics/settings.py


### 5. run 실행 파일 생성
- -o 파일이름.csv
- default = -a page=1

In [228]:
%%writefile run.sh
scrapy crawl Comics -o dc_comics_191211.csv -a page=25

Writing run.sh
